Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
import seaborn as sns
import re
import num2words
%matplotlib inline
pd.options.display.max_colwidth = 750

In [2]:
# You may need to change the path
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [3]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133",1
1,3861,"\nAn uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but with a pleasant minty coolness. (Wyoming only)",0
2,655,"\nThis release is a port version of Amrut’s Intermediate Sherry — a sort of port pipe sandwich. The spirit is matured in both unused casks and bourbon casks, then spends a few months in port pipes, and then returns to bourbon casks. The result is a Pink Floyd show of a whisky: vibrant, colorful, complex, and nearly too much. A blackcurrant and wispy, smoky nose gives way to an intense and bittersweet mix of chili, blackcurrant, oak, damson, dark chocolate, and peat. Astounding.",1
3,555,"\nThis 41 year old single cask was aged in a sherry butt and interacted magnificently with that wood. The nose offers peeled red apples, sultanas, honey, fudge, milk chocolate, and American cream soda. Balanced and harmonious throughout, the palate yields remarkably fresh orchard fruits, pipe tobacco, black pepper, and ginger. Long in the finish, with cinnamon drying to dark chocolate, and slightly smoky, unobtrusively tannic oak. A 1970s classic!",1
4,1965,"\nQuite herbal on the nose, with aromas of dried tarragon, parsley, and dill, along with cherry, milk chocolate, orange blossom, vanilla bean, and sweet tea. The palate is slightly grain-forward, but is full and well balanced, with orange, dried flowers, chamomile tea, lemon rind, and ginger. The finish offers cinnamon and milk chocolate against a note of candied lemon. Youthful and barrel-influenced, but admirably executed.",1


In [4]:
train.shape

(4087, 3)

In [5]:
target = 'ratingCategory'
features = 'description'
X = train[features]
y = train[target]

In [6]:
X.shape

(4087,)

In [7]:
y.shape

(4087,)

In [8]:
y

0       1
1       0
2       1
3       1
4       1
       ..
4082    1
4083    1
4084    1
4085    1
4086    1
Name: ratingCategory, Length: 4087, dtype: int64

In [9]:
assert len(X) == len(y)

In [10]:
X

0       \nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only) A$133
1                                                 \nAn uncommon exclusive bottling of a 6 year old cask strength malt. Light gold in color, the nose is vegetal, more peat bog than peat smoke, with an undercurrent of pastry cream and rose. It’s an odd combination of aromas. The entry is flavorful and inviting with smoked pineapple, clove, and rose. Peak smoke arrives in full force in the mid-palate, which drops the sweet and becomes spicy. The finish is mostly smoke, but wi

In [11]:
def clean_data(text):
    """
    Accepts a single text document and performs several regex substitutions in order to clean the document. 
    Parameters
    ----------
    text: string or object 
    Returns
    -------
    text: string or object
    """
    # order of operations - apply the expression from top to bottom
    email_regex = r"From: \S+@\S+"
    non_alpha = '[^a-zA-Z0-9]'
    multi_white_spaces = "[ ]{2,}"
    text = re.sub(email_regex, "", text)
    text = re.sub(non_alpha, ' ', text)
    text = re.sub(multi_white_spaces, " ", text)
    # To convert numbers such as, 1970s, into text. I believe these will meaningfully impact the accuracy of our models
    text = re.sub(r"(\d+)", lambda x: num2words.num2words(int(x.group(0))), text)
    # apply case normalization 
    return text.lower().strip()
# If I create another version of this for the countvectorizer as opposed to the tfidvectorizer, 
# I'll have to tokenize the text. And in doing so, I can use my list comphrehension to remove any and all
# empty strings.

In [12]:
X = X.apply(clean_data)

In [13]:
X

0       sometimes when whisky is batched a few leftover barrels are returned to the warehouse canadian club recently pulled and vatted several of these from the one thousand, nine hundred and seventys acetone granny smith apples and fresh cut white cedar showcase this long age complex and spicy yet reserved this dram is ripe with strawberries canned pears cloves pepper and faint flowers then slightly pulling oak tannins distinct elegant and remarkably vibrant this ancient canadian club is anything but tired australia only a one hundred and thirty-three
1                                                                                                     an uncommon exclusive bottling of a six year old cask strength malt light gold in color the nose is vegetal more peat bog than peat smoke with an undercurrent of pastry cream and rose it s an odd combination of aromas the entry is flavorful and inviting with smoked pineapple clove and rose peak smoke arrives in full force in the mid pala

### Define Pipeline Components

In [14]:
vect = TfidfVectorizer()
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [15]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(
                           pipe,
                           parameters,
                           cv=5,
                           n_jobs=4,
                           verbose=1
                           )

grid_search.fit(X,y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=4,
             param_grid={'clf__max_depth': (5, 10, 15, 20),
                         'vect__max_df': (0.75, 1.0)},
             verbose=1)

In [16]:
grid_search.best_params_

{'clf__max_depth': 20, 'vect__max_df': 0.75}

In [17]:
grid_search.best_score_

0.7117694589005634

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [18]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [19]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [20]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [21]:
subNumber = 0

In [22]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [32]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier

svd = TruncatedSVD()
vect = TfidfVectorizer(stop_words="english")
clf = XGBClassifier()

lsi = Pipeline([('vect', vect),
                ('svd', svd)])

pipe = Pipeline([('lsi', lsi),
                 ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [33]:
%%time

parameters = {
    'lsi__svd__n_components': [10,100,1000],
    'lsi__vect__max_df': (0.75, 1.0, 1.25),
    'clf__max_depth':(5,10,20,40)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=2)
grid_search.fit(X,y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


KeyboardInterrupt: 

In [34]:
grid_search.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
grid_search.best_score_

In [ ]:
best_xgb_model = grid_search.best_estimator_
best_xgb_model

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = best_xgb_model.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
from spacy.tokenizer import Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
def get_lemmas(text):

    lemmas = []
    
    doc = nlp(text)
    
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False) and (token.is_space == False)):
            lemmas.append(token.lemma_)
    
    return lemmas

In [ ]:
X_lemmas = X.apply(get_lemmas)
X_lemmas

In [ ]:
def listintostring(string):
    '''
    This function converts lists into strings
    '''
    str = " "
    return (str.join(string))

In [ ]:
X_lemmas_str = X_lemmas.apply(listintostring)

In [ ]:
STOP_WORDS = nlp.Defaults.stop_words

In [ ]:
tokens = []

for doc in tokenizer.pipe(X_lemmas_str):

    doc_tokens =[]

    for token in doc:
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower())
    tokens.append(doc_tokens)
    
X_tokens = tokens

In [ ]:
X_tokens

In [ ]:
# Moving training labels to array
y_array = train.ratingCategory.values

In [ ]:
# Apply to your Dataset

from scipy.stats import randint

param_dist = {
    
    'lsa__svd__n_components': [10,100,1000],
    'lsa__vect__max_df': (0.75, 1.0, 1.25),
    'clf__n_estimators':[10, 100, 10],
    'clf__max_depth' : randint(5,10,20,40),
    'clf__min_samples_leaf': randint(2,15,2)
    }

In [ ]:
%%time
# Continue Word Embedding Work Here

#SHOULD DO MORE EDA ON DATASET TO FIGURE HOW TO TUNE PARAMETERS


svd = TruncatedSVD()
gbc = GradientBoostingClassifier()
vect = TfidfVectorizer(stop_words="english")

lsa = Pipeline([("vect", vect),
                ("svd", svd)])

pipe = Pipeline([("lsa", lsa),
                 ("clf", gbc)])

rscv = RandomizedSearchCV(
                          pipe,
                          param_dist,
                          n_iter=5,
                          cv=27,
                          n_jobs=-1,
                          verbose=1
                          )

rscv.fit(X_lemmas_str,y_array)

In [ ]:
rscv.best_params_

In [ ]:
rscv.best_score_

In [ ]:
# Use an XGBoost Classifier model after this

In [ ]:
# What are the best metrics to use to understand how well our models are performing?

# This is a classification task so the usual ones are fine: Accuracy, Precision, Recall, F1

#*I'm also using classification report, roc curve, confusion matrices, and this:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html

### Make a Submission File

In [ ]:
# Predictions on test sample
pred = rscv.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?